**Show all your work for full credit. Each source code you submit should include detailed comments and instructions on how to run it in order to confirm that it works as expected. If the program that does not run or throws runtime errors, it cannot be graded. You can refer to the programming guidelines from the TAs here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

**This is an individual assignment and each student should work on their own. Ensure you don't share any code online or with others (note, using Replit, GitHub and similar online platforms can make your code accessible to others).**

**To submit the assignment, you need to use Jupyter Notebook with the provided cell blocks and follow the naming conventions and instructions posted here: https://tinyurl.com/CPEG-472-672-Programming-Guide/**

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and section below:

In [3]:
NAME = "Shruthilaya Arun"
#SECTION = "472"
SECTION = "672"

---

## <font color='blue'> Q3: [40 points total - answer all parts] </font>


### <font color='blue'> Q3-a: [10 points] Implement both the encryption and decryption operation for a block cipher called BH (BlueHen) that is based on a Feistel network. Assume BH has a block size of 128 bits, its symmetric key is 128 bits and the number of rounds is 5. For the PRF of the Feistel network of BH you should adopt the GGM89 construction (see week3_GGM_PRF.zip under Files => Code Samples). What is the size of each subkey of BH? How many different subkeys are needed in the Feistel network of BH? For key expansion, use a secure PRNG seeded with the 128-bit key of BH to generate all required subkeys. Demonstrate correct operation of BH encryption and decryption using 2 ciphertext/plaintext pairs. **Note, this sub-question focuses on the block cipher itself, not any modes of operation for BH. </font>


In [4]:
# Import the libraries here ...
import secrets

In [5]:
# Code from https://github.com/davidlazar/python-drbg
# Implements HMAC_DRBG (SHA-512) from NIST SP 800-90A

import hashlib
import hmac

class DRBG(object):
    def __init__(self, seed):
        self.key = b'\x00' * 64
        self.val = b'\x01' * 64
        self.reseed(seed)

    def hmac(self, key, val):
        return hmac.new(key, val, hashlib.sha512).digest()

    def reseed(self, data=b''):
        self.key = self.hmac(self.key, self.val + b'\x00' + data)
        self.val = self.hmac(self.key, self.val)

        if data:
            self.key = self.hmac(self.key, self.val + b'\x01' + data)
            self.val = self.hmac(self.key, self.val)

    def generate(self, n):
        xs = b''
        while len(xs) < n:
            self.val = self.hmac(self.key, self.val)
            xs += self.val

        self.reseed()

        return xs[:n]

In [6]:
# you can use this function to xor two bytes, or two strings.
def xor(x,y):
    if type(x) == type(y) == str:
        return "".join([chr(ord(a) ^ ord(b)) for a,b in zip(x,y)])
    elif type(x) == type(y) == bytes:
        return "".join([chr(a ^ b) for a,b in zip(x,y)]).encode('iso-8859-1')
    else:
        print("Error: type mismatch")

# [1 points] What is the size of each subkey of BH? How many different subkeys are needed in the Feistel network of BH?

The size of each subkey is 16 bytes and 5 subkeys are needed for the 5 rounds of the Feistal network.

In [7]:
def generate_key() -> bytes:
    """
    generate random 128 bit key. You can use secrets library
    """
    key = b""
    # YOUR CODE HERE
    key=secrets.token_bytes(16) # generate 16 bytes
    #raise NotImplementedError()
    return key
    
# 1 points
def key_expansion(key: bytes)-> list:
    """
    Implement the key_expansion based on the blueprint asked in the question. You can
    use the DRBG class provided as a secure PRNG. The input key should be 128 bits.
    Return subkeys which is a list of bytes.
    """
    subkeys = []
    # YOUR CODE HERE
    drbg=DRBG(key)  
    expanded_key=drbg.generate(80)  # generate 80 bytes
    for i in range(0, 80, 16):
        subkey=expanded_key[i:i+16] # divide the 80 bytes among 5 subkeys of 16 bytes each
        subkeys.append(subkey) # add to list
    return subkeys
# GGM89
def PRG(seed):
    assert isinstance(seed, bytes)
    drbg = DRBG(seed)
    rnd = drbg.generate(2*len(seed))
    return rnd[:len(seed)], rnd[len(seed):]

def PRF(key, msg):
    assert isinstance(key, bytes)
    assert isinstance(msg, bytes)
  # convert data into sequence of bits
    msgBits = "".join(format(byte, '08b') for byte in msg)
    seed = key # initialize the seed with the key
    for b in msgBits:
        rnd = PRG(seed)[int(b)]
        seed = rnd
    return rnd
    
# 2 points
def feistel_function(ptxt: bytes, subkey: bytes) -> bytes:
    """
    Based on the blueprint given in the question, implement the feistel function
    and generate random_string and return it.
    """
    random_string = b""
    # YOUR CODE HERE
    random_string = PRF(subkey, ptxt) # Use GGM89
    #raise NotImplementedError()
    return random_string

# 2 points
def BH_encrypt(ptxt: bytes, key: bytes = b"") -> bytes:
    """
    Implement the BlueHen encryption function. The function takes the plaintext
    and key in bytes and returns the ciphertext in bytes format.
    You must call the key_expansion and feistel_function functions.
    """
    if key == b"":
        key = gen_key()
    ctxt = b''
    # YOUR CODE HERE
    original_length=len(ptxt)
    if original_length % 2 != 0: 
        ptxt += b'\x00'# add padding if not even length
    left=ptxt[:len(ptxt) // 2] # divide ptxt into left and right half
    right=ptxt[len(ptxt) // 2:]
    subkeys=key_expansion(key) # get subkeys

    for i in range(len(subkeys)):
        #feistel network
        feistel_result = feistel_function(right, subkeys[i]) 
        left, right = right, xor(left, feistel_result)

    ctxt= left + right # add both halves to get ctxt
    return ctxt

# 2 points
def BH_decrypt(ctxt: bytes, key: bytes = b"") -> bytes:
    """
    Implement the BlueHen encryption function. The function takes the ciphertext
    and key in bytes and returns the plaintext in bytes format.
    You must call the key_expansion and feistel_function functions.
    """
    if key == b"":
        key = gen_key()
    ptxt = b''

    # YOUR CODE HERE
    left=ctxt[:len(ctxt) // 2]
    right=ctxt[len(ctxt) // 2:]
    subkeys=key_expansion(key)
    # reverse order for decryption
    for i in range(len(subkeys) - 1, -1, -1):
        feistel_result = feistel_function(left, subkeys[i])
        right, left = left, xor(right, feistel_result)

    ptxt=left + right
    return ptxt.rstrip(b'\x00') # remove padding
    return ptxt


### [1 points] Demonstrate correct operation of BH encryption and decryption using 2 ciphertext/plaintext pairs.

In [8]:
ptxt = b"myname"
key = generate_key()
ctxt = BH_encrypt(ptxt, key)
print(key)
ptxt_test = BH_decrypt(ctxt, key)
print(ptxt,ptxt_test)
assert ptxt == ptxt_test


ptxt = b"hello"
key = generate_key()
ctxt = BH_encrypt(ptxt, key)
print(key)
ptxt_test = BH_decrypt(ctxt, key)
print(ptxt,ptxt_test)
assert ptxt == ptxt_test


b'd\xf5=\x0c\x8dz);&\\\xae7\xe8\x14\xc4\xde'
b'myname' b'myname'
b'\xf9\xe9\xe8sC o\xce,\xbc/S\xab\xc6\x04\xb4'
b'hello' b'hello'


In [9]:
key = generate_key()
assert len(key) == 16
assert type(key) == bytes

In [10]:
key = b"this is a test!!"
subkeys = key_expansion(key)
assert subkeys == [b'\x0e\xaeW\x08\xc3*\x83\xc3d\xe8Y\xf7\xbfr\x07\xb0', b"\xb6\xa0,jg\xf4;\xda\xb6\xbeV\x05'\x93if", b'9\x16\xbf*\xed\xbbs\xd4\xe8\xf8w6Fs\xb4\xe6', b'\xa6\xa5\xc7\xbc|\xf92\x1fy\xfev\x9f\x12\xbeT\x80', b'\xf1:\x18i\xac\x97%\x97\\\xd5\x9c\x14\xdc\xf6\x90\xcb']


In [11]:
random_string = feistel_function(b"hello",b"this is a test!!")
assert random_string == b'\x07\x8c\x90\xad\xfeP[\x1e\xd0Xn-vf\x88>'

In [12]:
# hidden tests

In [13]:
k1 = b'O\xce)m\xb8\xb3K\xfa\xb6J%W\x87\n\xab\xf9'
p1 = b'abcdef  abcdef'
c1 = BH_encrypt(p1, k1) 
assert c1 == b'z\xeb\xb4\x04\x17\xbdk\xb3X\xa0\x8a\xbf\xc4\xf1' 

In [14]:
# hidden tests

In [15]:
k1 = b'O\xce)m\xb8\xb3K\xfa\xb6J%W\x87\n\xab\xf9'
c1 = b'z\xeb\xb4\x04\x17\xbdk\xb3X\xa0\x8a\xbf\xc4\xf1'
p1 = BH_decrypt(c1, k1) 
assert p1 == b'abcdef  abcdef'

In [16]:
# hidden test cases

### <font color='blue'> Q3-b: [10 points] Implement the counter (CTR) mode decryption for the BH block cipher described above. Assume nonce size of 64 bits and counter size of 64 bits. Ensure the counter restarts from zero in every new message. Do you need a different nonce for each message or you can use the same? What are the inputs needed to implement BH-CTR decryption? Demonstrate correct operation of BH-CTR decryption using a ciphertext comprising at least 4 blocks and any other necessary inputs. Do you need to use padding in this mode of operation? </font>

### [3 points] Do you need a different nonce for each message or you can use the same? What are the inputs needed to implement BH-CTR decryption? Do you need to use padding in this mode of operation?

A different nonce is required for each message, you cannot use the same nonce for different messages. 
The inputs needed for the BH_CTR decryption are the nonce, key and the ciphertext. 
Counter mode does not require padding.

In [17]:
# 2 points
def generate_nonce() -> bytes:
    """
    Must generate 64 bit random nonce. You can use secrets library.
    """

    nonce = b""
    
    # YOUR CODE HERE
    nonce=secrets.token_bytes(8)
    #raise NotImplementedError()

    return nonce

# 5 points
def BH_CTR_decrypt(ctxt: bytes, key: bytes = b"", nonce: bytes = b"") -> bytes:
    """
    Implement the CTR mode decryption of BH cipher. Input ctxt should be at least
    4 blocks, or 64 bytes"
    The function should return the result as bytes.
    You must utilize the previous functions implemented in part a.
    """
    if key == b"":
        key = generate_key()
    if nonce == b"":
        nonce = generate_nonce()
    
    ptxt = b''
    counter = 0
    # YOUR CODE HERE
    block_size=16 # AES block size
    for i in range(0,len(ctxt),block_size):
        counter_block=nonce+counter.to_bytes(8,byteorder='big')# concatinate nonce and counter 
        encryption=BH_encrypt(counter_block,key) # encrypt contantinated counter and nonce with key
        current_block=ctxt[i:i+block_size] # get current_block
        decrypted_block=xor(encryption,current_block)#decrypted block
        ptxt += decrypted_block
        counter += 1
    return ptxt

In [18]:
nonce = generate_nonce()
assert len(nonce) == 8
assert type(nonce) == bytes

In [19]:
txt1 = b'abcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdabcdab'
key1 = b'\xaf;S"\x06\xca\x1b\xeb]\x85\xab :^\xf8\x86'
nonce1 = b'\x98\xae\xbfR\x9c[4\xb2'
res1 = BH_CTR_decrypt(txt1, key1, nonce1)
assert res1 == b'\xaa\x8do]\r\xddo~\x883t\xf4\xab\xdc\x94\xdd\xa9\xc3\xe2\xd9\xe1\x03\x06\xab&\xa3\x9f\xe5i\x9eKM\xab\xd3\x0brq\xc3;\x8b}\xc9\x89\xa7\xff\xdbY2\xe2\xe9t\xe9 \x1f\xbct"zU\xcd\x9dt\xe7\xe8\xe6\xb3\x87]%\x90\x87\x88\xd78'
assert txt1 == BH_CTR_decrypt(res1, key1, nonce1)

txt2 = b'Hello World, this is applied cryptography. Welcome to a world full of encryption'
key2 = b'\xbe\t/!\xe2qz\xad\x10DhQ\x90l\xd3i'
nonce2 = b'\xd8^\xceIc\xc0\xe0\xf6'
res2 = BH_CTR_decrypt(txt2, key2, nonce2)
assert res2 == b'\xa8#9\x8eFda\xa4\x94EY\x8e(Q32a/\xbd\x90\xeef\xb5\xfa\x14\x03X\xe8%\xe0\x05\xa3\x89\xe8\xb0:\xf21\x87\xb9)\xc0\xe3\x90\\\xdc?\xf7Z\xeb\xaf\xa5G\x95\x91i\xcc\x90\xf7\\.\xe3\xa0\xcaAo\x8fka\\-\x82e\xa3\xf2\x1e\xf5,\xff\xe1'
assert txt2 == BH_CTR_decrypt(res2, key2, nonce2)

In [20]:
# hidden test cases

### <font color='blue'> Q3-c: [10 points] Implement the cipher block chaining (CBC) mode encryption for the BH block cipher described above. What are the inputs needed to implement BH-CBC decryption? Demonstrate correct operation of BH-CBC encryption using a message of size equal to 3 blocks and any other necessary inputs. **Note, no need to worry about plaintext alignment to the block size in this sub-question.
</font>

### [2 points] What are the inputs needed to implement BH-CBC decryption?

The inputs needed are the ciphertext,the IV and the key.

In [21]:
# 2 points
def generate_iv() -> bytes:
    """
    generate 128 bit random IV. You can use secrets library.
    """

    iv = b""
    # YOUR CODE HERE
    iv=secrets.token_bytes(16)
    #raise NotImplementedError()

    return iv
def pad(ptxt,block_size):
    padding_length = block_size - (len(ptxt) % block_size) # calculate required padding length
    padding = bytes([padding_length] * padding_length)  # get the byte to add and number of bytes to add
    return ptxt + padding

# 6 points
def BH_CBC_encrypt(ptxt: bytes, key: bytes = b"", iv: bytes = b"") -> bytes:
    """
    Implement the BH CBC encryption mode.
    The function should return ctxt as bytes.
    You must utilize the previous functions implemented in part a.
    """

    if key == b"":
        key = generate_key()

    if iv == b"":
        iv = generate_iv()

    ctxt = b""
    # YOUR CODE HERE
    block_size=16
    padded_ptxt=pad(ptxt,block_size) # add padding
    previous_ctxt=iv # for 1st block
    for i in range(0,len(padded_ptxt),block_size):
        block=padded_ptxt[i:i+block_size] # get the blocks
        xor_block=xor(block,previous_ctxt) # xor input for encryption
        encryption=BH_encrypt(xor_block,key) # encrypting xor_bloxk and key
        ctxt += encryption
        previous_ctxt=encryption
    
    #raise NotImplementedError()

    return ctxt

In [22]:
iv = generate_iv()
assert len(iv) == 16
assert type(iv) == bytes

In [23]:
ptxt1 = b'abcdabcdabcdabcdabcdabcdabcdabcdcdabcdabcdabcdbcdabcd'
iv1 = b'\xe0\xc1\x81\xfbJ]\x02\x16V\x99\xef!\xef+\xf0\xdb'
key1 = b'\xda\x10\x82\xa2\x15\xa9\xf4\xbdt"\xae\x98NJ#\x0c'
ctxt1 = BH_CBC_encrypt(ptxt1, key1, iv1)
assert ctxt1 == b'\xf2\xef\x1e\x9b\xaaA$\xd4\xed\x00b\xcb\x05\xfb\x91K\x0b\xf1\xfaD\xb1|\x0eJ!\xf6\xfc\x1d_~\xb3l\xef\x1ah\xff\xf9s\xd0\xd5Qa\xfa>&\xd6\xef\x8ck\xb0y(h'

ptxt2 = b'Crypto is AWESOME Crypto is AWESOME Crypto is AWESOME Crypto is AWESOME Crypto is AWESOME Crypto is AWESOME Crypto is AWESOME Crypto is AWESOME '
iv2 = b'9\x16\x03\xcf\xcf\xacP\xc3\x99\r\x15y\xf0\xca\x1b\xae'
key2 = b'1\xe2\xf0s\x9fh#\xd42\x0f>\xb2\xcd\r\xd9~'
ctxt2 = BH_CBC_encrypt(ptxt2, key2, iv2)
assert ctxt2 ==  b'\xd6\x85\x96\xd0\x15V\x81V[\xe9\x0e\xc8\xb6\x82\x13\x9c\x135]\xde\xc6\x99\xd6\xfd\x0c\xc6\x80=0eC\xfe^\xda\xe3\x07\x1a-\xf0V\xfb|\x03_\xe8\xe1\x01\xac\x19\x8aK\xfe\xee\xdf3"h\x89\x96\xf2\x1f\x99\xfd~\x00\x05A\xa0\x07\x00c\xfb\x8a\x04\xdez\x83\x06\xe1f}_.P\xc2\xd1\xfa%\xb6\xca4Bz\x0e\xc8A\x02\x9e\xc45h:)\x03D<\xe1\xe6\xb8\x14\x04\x81\xf4\xc2\xd4F\xd2\xf6\xa1\x85R\xeb\x0b\x88\x1bY\xd0\xd2\x16\x02\xf3v\x03!\xc1?m\x10\x96\x1dB\x12w{'


AssertionError: 

In [24]:
# hidden test cases

### <font color='blue'> Q3-d: [10 points] For the BH-CBC mode above, assume the input message does not align to a multiple of the block size. Extend BH-CBC to BH-CBC-CS that supports ciphertext stealing, as discussed in class and in the book. Implement the BH-CBC-CS encryption and decryption functions. Demonstrate correct operation by encrypting a 40-byte plaintext message with BH-CBC-CS and then by correctly decrypting the corresponding 40-byte ciphertext.
</font>

In [29]:
# worth 10 points
def BH_CBC_CS_encrypt(ptxt: bytes, key: bytes = b"", iv: bytes = b"") -> bytes:
    """
    Implement the encryption function of CBC mode that supports ciphertext stealing. Implement based
    on the blueprint provided in the lecture. return ctxt in bytes
    """

    if key == b"":
        key = generate_key()

    if iv == b"":
        iv = generate_iv()

    ctxt = b''
    # YOUR CODE HERE
    block_size=16
    no_of_block=len(ptxt)//block_size
    remainder=len(ptxt)%block_size
    if remainder == 0:
        return BH_CBC_encrypt(ptxt,key,iv)
    previous_ctxt=iv
    for i in range(no_of_block-1):
        block=ptxt[i*block_size:(i+1)*block_size]
        xor_block=xor(block,previous_ctxt) # xor input for encryption
        encryption=BH_encrypt(xor_block,key) # encrypting xor_bloxk and key
        ctxt += encryption
        previous_ctxt=encryption
        
    penultimate_block=ptxt[(no_of_block-1)*block_size:no_of_block*block_size] #getting the penultimate block
    final_block=ptxt[no_of_block*block_size:] # getting last block
    # encrypting penultimate block
    xor_block=xor(penultimate_block,previous_ctxt) 
    encrypted_penultimate_block=BH_encrypt(xor_block,key)
    # getting the remaining from the penultimate block
    stolen_ctxt= encrypted_penultimate_block[:remainder]
    xor_final_block=xor(final_block,stolen_ctxt) # xor with the final block
    encrypted_final_block=BH_encrypt(xor_final_block,key) # encrypt final block
    ctxt += encrypted_final_block + encrypted_penultimate_block[:remainder]

    
    #raise NotImplementedError()

    return ctxt

In [30]:
ptxt = b'Say my name then write it on a paper and store it forever'
iv = b'\xe0\xc1\x81\xfbJ]\x02\x16V\x99\xef!\xef+\xf0\xda'
key = b'\xda\x10\x82\xa2\x15\xa9\xf4\xbdt"\xae\x98NJ#\x0c'
ctxt = BH_CBC_CS_encrypt(ptxt,key,iv)
assert ctxt == b'\xca\xb5\xe5\xb0k\xdcv\xa2,\x99\x07pCP\x81\xb4\xd3\x8aT\xa7\xa3\xba\x1d\x7f\xb4\xf5\x03\xbc\xe8]+\xbe\xb7q\xbe\xde\xd5\x12\xd6\xa0\xb1\x11\xac1u/\xcd\x17\xf9\x07\xdf\xe78\x18U\xed{'

ptxt2 = b'crypto <3 crypto <3 crypto <3 crypto <3 crypto <3 crypto <3 crypto <3 crypto <3'
iv2 = b'\x155F\x84m\x0e\xb5\x18\x14W\xf0\xa6N\x85\xab\x00'
key2 = b'\x1e\xc9\xb3\x9aU\xf6p\xda,\xae\x8a\x8b\xa0Q\xff\xb2'
ctxt2 = BH_CBC_CS_encrypt(ptxt2, key2, iv2)
print(ctxt2)
assert ctxt2 == b"\xc4\x8ft\xfc\xabBo\x84\xb4\x03\x98|'e:\xfc\x89N\x90\xe2\xcc\x18p\xce#\xcc\xa1\xf0\x03A5m\xe9\x9fSjT\xd6][ \x87yR4\xb7i+b,2\xc8\xcc\xa9zy-\xf9Q\xc9\x93\xc4\x9epwW:\x005}\x08('\x83MjK\xb4\xe0"


AssertionError: 

In [ ]:
# hidden tests